In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE45032"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE45032"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE45032.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE45032.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE45032.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression in liver of HCC and CHC patients"
!Series_summary	"In order to compare age depenpdence of mRNA between HCC and CHC patients, we measured gene expression by microarray."
!Series_overall_design	"24 liver samples are taken from HCC and CHC patients with various ages and gender."
Sample Characteristics Dictionary:
{0: ['cell type: hepatocallular carcinoma', 'cell type: chronic hepatitis type C'], 1: ['tissue: liver'], 2: ['gender: male', 'gender: female'], 3: ['age(yrs): 67', 'age(yrs): 56', 'age(yrs): 76', 'age(yrs): 79', 'age(yrs): 66', 'age(yrs): 70', 'age(yrs): 68', 'age(yrs): 72', 'age(yrs): 62', 'age(yrs): 55', 'age(yrs): 71', 'age(yrs): 73', 'age(yrs): 74', 'age(yrs): 61', 'age(yrs): 54', 'age(yrs): 64', 'age(yrs): 59', 'age(yrs): 69', 'age(yrs): 25', 'age(yrs): 41', 'age(yrs): 50', 'age(yrs): 58', 'age(yrs): 49', 'age(yrs): 63', 'age(yrs): 60', 'age(yrs): 52', 'age(yrs): 51']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series_title and Series_summary, this appears to be a microarray gene expression dataset
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Liver_Cancer): From the sample characteristics, key 0 has "cell type: hepatocallular carcinoma" and "cell type: chronic hepatitis type C"
trait_row = 0

# For age: From the sample characteristics, key 3 has age information
age_row = 3

# For gender: From the sample characteristics, key 2 has gender information
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert cell type values to binary trait values (HCC=1, CHC=0)"""
    if isinstance(value, str):
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "hepatocallular carcinoma" in value.lower() or "hcc" in value.lower():
            return 1  # HCC = 1 (Liver Cancer)
        elif "chronic hepatitis" in value.lower() or "chc" in value.lower():
            return 0  # CHC = 0 (not Liver Cancer but Hepatitis)
    return None

def convert_age(value):
    """Convert age values to continuous values"""
    if isinstance(value, str):
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        # Extract digits only to handle potential format like "age(yrs): 67"
        age_str = ''.join(c for c in value if c.isdigit())
        if age_str:
            return float(age_str)
    return None

def convert_gender(value):
    """Convert gender values to binary (female=0, male=1)"""
    if isinstance(value, str):
        if ":" in value:
            value = value.split(":", 1)[1].strip().lower()
        else:
            value = value.lower()
        
        if "female" in value or "f" == value:
            return 0
        elif "male" in value or "m" == value:
            return 1
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the library function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save clinical data to CSV file
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM1096016': [1.0, 67.0, 1.0], 'GSM1096017': [1.0, 56.0, 1.0], 'GSM1096018': [1.0, 76.0, 0.0], 'GSM1096019': [1.0, 79.0, 1.0], 'GSM1096020': [1.0, 66.0, 1.0], 'GSM1096021': [1.0, 70.0, 1.0], 'GSM1096022': [1.0, 68.0, 1.0], 'GSM1096023': [1.0, 72.0, 0.0], 'GSM1096024': [1.0, 62.0, 1.0], 'GSM1096025': [1.0, 66.0, 1.0], 'GSM1096026': [1.0, 55.0, 1.0], 'GSM1096027': [1.0, 62.0, 1.0], 'GSM1096028': [1.0, 71.0, 1.0], 'GSM1096029': [1.0, 73.0, 0.0], 'GSM1096030': [1.0, 74.0, 0.0], 'GSM1096031': [1.0, 61.0, 0.0], 'GSM1096032': [1.0, 54.0, 1.0], 'GSM1096033': [1.0, 64.0, 0.0], 'GSM1096034': [1.0, 68.0, 1.0], 'GSM1096035': [1.0, 59.0, 1.0], 'GSM1096036': [1.0, 79.0, 1.0], 'GSM1096037': [1.0, 69.0, 0.0], 'GSM1096038': [1.0, 59.0, 1.0], 'GSM1096039': [1.0, 71.0, 1.0], 'GSM1096040': [0.0, 64.0, 1.0], 'GSM1096041': [0.0, 55.0, 0.0], 'GSM1096042': [0.0, 66.0, 0.0], 'GSM1096043': [0.0, 56.0, 1.0], 'GSM1096044': [0.0, 66.0, 1.0], 'GSM1096045': [0.0, 68.0, 1.0], 'GSM1096046':

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_Cancer/GSE45032/GSE45032_series_matrix.txt.gz


Gene data shape: (62976, 48)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers in the gene expression data.
# The identifiers appear to be simple numeric indices (e.g. '1', '2', '3') rather than 
# actual human gene symbols like 'BRCA1', 'TP53', etc. or other typical gene identifiers 
# such as Entrez IDs or Ensembl IDs.
#
# In order to perform meaningful biological analysis, these numeric identifiers will need
# to be mapped to standard human gene symbols or other recognized gene identifiers.
# This likely means we need to find a mapping file or reference that connects these
# numeric indices to gene symbols, possibly in the platform annotation file.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'ProbeName', 'GB_ACC', 'ControlType', 'accessions', 'GeneName', 'Description', 'chr_coord', 'SEQUENCE', 'SPOT_ID']
{'ID': ['1', '2', '3', '4', '5'], 'ProbeName': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P326296', 'A_24_P287941'], 'GB_ACC': [nan, nan, nan, 'NM_144987', 'NM_013290'], 'ControlType': [1.0, 1.0, 1.0, 0.0, 0.0], 'accessions': [nan, nan, nan, 'ref|NM_144987|ref|NM_001040425|ens|ENST00000292879|ens|ENST00000392196', 'ref|NM_013290|ref|NM_016556|ens|ENST00000393795|ens|ENST00000253789'], 'GeneName': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'U2AF1L4', 'PSMC3IP'], 'Description': [nan, nan, nan, 'ref|Homo sapiens U2 small nuclear RNA auxiliary factor 1-like 4 (U2AF1L4), transcript variant 2, mRNA [NM_144987]', 'ref|Homo sapiens PSMC3 interacting protein (PSMC3IP), transcript variant 1, mRNA [NM_013290]'], 'chr_coord': [nan, nan, nan, 'hs|chr19:036235296-036235237', 'hs|chr17:040724775-040724716'], 'SE

### Step 6: Gene Identifier Mapping

In [7]:
# Analyzing the gene annotation dataframe to identify mapping columns
# The 'ID' column in gene_annotation appears to match with the row IDs in gene_data
# The 'GeneName' column appears to contain the gene symbols we want to map to

# 1. Create a mapping dataframe with columns 'ID' and 'Gene'
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GeneName')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 2. Apply the gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data after mapping, shape: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# 3. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data after normalization, shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# 4. Save the processed gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (62976, 2)
First few rows of mapping dataframe:
  ID             Gene
0  1  GE_BrightCorner
1  2       DarkCorner
2  3       DarkCorner
3  4          U2AF1L4
4  5          PSMC3IP


Gene expression data after mapping, shape: (20147, 48)
First few gene symbols after mapping:
['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AA081107', 'AA213559']


Gene expression data after normalization, shape: (19274, 48)
First few normalized gene symbols:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Gene expression data saved to ../../output/preprocess/Liver_Cancer/gene_data/GSE45032.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data loaded from file:")
print(preview_df(clinical_df))

# 2. Normalize gene symbols 
# Note: This was already done in a previous step, but we'll ensure gene_data has been normalized
print("\nVerifying gene data normalization...")
print(f"Gene data shape: {gene_data.shape}")
print("First 10 gene identifiers:")
print(gene_data.index[:10].tolist())

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    preview_cols = list(linked_data.columns[:5])
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
# First check how many samples have missing trait values
if trait in linked_data.columns:
    missing_trait = linked_data[trait].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Apply missing value handling
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This dataset contains liver tissue gene expression from hepatocellular carcinoma (HCC) and chronic hepatitis C (CHC) patients with various ages and gender."
is_gene_available = len(gene_data) > 0
is_trait_available = trait in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=trait_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Clinical data loaded from file:
{'GSM1096016': [1.0, 67.0, 1.0], 'GSM1096017': [1.0, 56.0, 1.0], 'GSM1096018': [1.0, 76.0, 0.0], 'GSM1096019': [1.0, 79.0, 1.0], 'GSM1096020': [1.0, 66.0, 1.0], 'GSM1096021': [1.0, 70.0, 1.0], 'GSM1096022': [1.0, 68.0, 1.0], 'GSM1096023': [1.0, 72.0, 0.0], 'GSM1096024': [1.0, 62.0, 1.0], 'GSM1096025': [1.0, 66.0, 1.0], 'GSM1096026': [1.0, 55.0, 1.0], 'GSM1096027': [1.0, 62.0, 1.0], 'GSM1096028': [1.0, 71.0, 1.0], 'GSM1096029': [1.0, 73.0, 0.0], 'GSM1096030': [1.0, 74.0, 0.0], 'GSM1096031': [1.0, 61.0, 0.0], 'GSM1096032': [1.0, 54.0, 1.0], 'GSM1096033': [1.0, 64.0, 0.0], 'GSM1096034': [1.0, 68.0, 1.0], 'GSM1096035': [1.0, 59.0, 1.0], 'GSM1096036': [1.0, 79.0, 1.0], 'GSM1096037': [1.0, 69.0, 0.0], 'GSM1096038': [1.0, 59.0, 1.0], 'GSM1096039': [1.0, 71.0, 1.0], 'GSM1096040': [0.0, 64.0, 1.0], 'GSM1096041': [0.0, 55.0, 0.0], 'GSM1096042': [0.0, 66.0, 0.0], 'GSM1096043': [0.0, 56.0, 1.0], 'GSM1096044': [0.0, 66.0, 1.0], 'GSM1096045': [0.0, 68.0, 1.0], 'GSM109

Linked data shape after handling missing values: (48, 19277)

Checking for bias in dataset features...
For the feature 'Liver_Cancer', the least common label is '1.0' with 24 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Liver_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 56.0
  50% (Median): 62.5
  75%: 68.0
Min: 25.0
Max: 79.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 17 occurrences. This represents 35.42% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Liver_Cancer/GSE45032.csv
